# Debugging Bad Solutions: Module 1, Part 1

From Midterm 2, Spring 2023 - Easy Exercises

## Introduction

### Purpose

On the exams you may initially write solutions that do not pass the test cases. That's okay! You will need to debug your code to determine what is causing the issue(s) and then figure out to how fix them. So how can we get better at debugging? We practice!

Below are three 1-point exercises and one 2-point exercise from the Spring 2023 Midterm 2. We have pre-written solutions for each exercise that are "bad" in one or more ways. Our solutions may contain one or more logic and/or syntax errors. Can you find and fix the issues in each exercise and pass all of the test cases?

Right before each exercise test cell, there is a block of text explaining the variables available to you for debugging. You may use these to test your code and can print/display them as needed.

**Exercise point breakdown:**

- Exercise 1: **1** point
- Exercise 4: **1** point
- Exercise 7: **1** point
- Exercise 5: **2** points

### Task Background: Better Reads

[Goodreads](https://www.goodreads.com/) is a website devoted to curating user-generated book reviews. You'll do some elementary data-mining to uncover "communities" of users who like the same books. Such insights might help users find like-minded communities and generate better book recommendations.

### Environment Configuration

#### Required Files

Run the following cells to download the required files.

In [ ]:
# import files
!wget -nc https://raw.githubusercontent.com/gt-cse-6040/topic_07_MT2_SP23_1467/main/demo_ex1.db
!wget -nc https://raw.githubusercontent.com/gt-cse-6040/topic_07_MT2_SP23_1467/main/demo_ex6.obj
!wget -nc https://raw.githubusercontent.com/gt-cse-6040/topic_07_MT2_SP23_1467/main/demo_ex7.db
!wget -nc https://raw.githubusercontent.com/gt-cse-6040/topic_07_MT2_SP23_1467/main/ex1-is_read.df
!wget -nc https://raw.githubusercontent.com/gt-cse-6040/topic_07_MT2_SP23_1467/main/ex1-is_reviewed.df
!wget -nc https://raw.githubusercontent.com/gt-cse-6040/topic_07_MT2_SP23_1467/main/goodreads.db
!wget -nc https://raw.githubusercontent.com/gt-cse-6040/topic_07_MT2_SP23_1467/main/tc_1
!wget -nc https://raw.githubusercontent.com/gt-cse-6040/topic_07_MT2_SP23_1467/main/tc_4
!wget -nc https://raw.githubusercontent.com/gt-cse-6040/topic_07_MT2_SP23_1467/main/tc_6
!wget -nc https://raw.githubusercontent.com/gt-cse-6040/topic_07_MT2_SP23_1467/main/tc_7
!wget -nc https://raw.githubusercontent.com/gt-cse-6040/topic_07_MT2_SP23_1467/main/ex1-rating.df
!wget -nc https://raw.githubusercontent.com/gt-cse-6040/topic_07_MT2_SP23_1467/main/ex1-user_id.df
!wget -nc https://raw.githubusercontent.com/gt-cse-6040/topic_07_MT2_SP23_1467/main/ex6-comms.df
!wget -nc https://raw.githubusercontent.com/gt-cse-6040/topic_07_MT2_SP23_1467/main/ex7-means.df
!wget -nc https://raw.githubusercontent.com/gt-cse-6040/topic_07_MT2_SP23_1467/main/ex7-sampler-input.df

!mkdir tester_fw
%cd tester_fw

!wget -nc https://raw.githubusercontent.com/gt-cse-6040/topic_07_MT2_SP23_1467/main/tester_fw/__init__.py
!wget -nc https://raw.githubusercontent.com/gt-cse-6040/topic_07_MT2_SP23_1467/main/tester_fw/test_utils.py
!wget -nc https://raw.githubusercontent.com/gt-cse-6040/topic_07_MT2_SP23_1467/main/tester_fw/testers.py

%cd ..

!wget -nc https://raw.githubusercontent.com/gt-cse-6040/bootcamp/mod1-debugging-dev/Module%201/Session%208/requirements.txt
!python -m pip install -r requirements.txt

#### Required Modules and Functions

Run the following cells to configure your environment.

In [ ]:
# Standard Python modules
import sys
import numpy as np
import pandas as pd
import sqlite3 as db
import math
import dill as pickle
from pprint import pprint, pformat
from tester_fw.db_utils import *

In case it's helpful, here are the versions of Python and standard modules you are using:

In [ ]:
print("* Python version: {}".format(sys.version.replace('\n', ' ')))
print(f"* Numpy version: {np.__version__}")
print(f"* pandas version: {pd.__version__}")
print(f"* sqlite3 version: {db.version}")

#### Creating the Required Database

Some of the Goodreads data is stored in a SQLite3 database. The code cell below opens a read-only connection to it named **`grdbconn`**.

For now, don't worry about what's there. We will explain any tables you need in the exercises that use them.

In [ ]:
# Goodreads database connection:
# grdbconn = db.connect('file:resource/asnlib/publicdata/goodreads.db?mode=ro', uri=True)
grdbconn = db.connect('file:goodreads.db?mode=ro', uri=True)

## **Ex. 1 (1 pt)**: `count_interactions_by`

### Background: Analyzing user-book interactions

The Goodreads dataset includes **user-book interactions.** An "user-book interaction" means the user "did something" with the book on the Goodreads website:

- _Viewed_: The user looked at a book description and saved it to their personal library.
- _Read_: The user marked the book as "read."
- _Rated_: The user gave the book a rating, from 1 to 5 stars.
- _Reviewed_: The user wrote a public review of the book on the website.

These interactions are recorded in a SQL table called `Interactions`. Let's have a quick look for one of the users whose integer ID is `874199`:

In [ ]:
pd.read_sql(r"SELECT * FROM Interactions WHERE user_id=874199", grdbconn)

Each row shows how this user interacted with some book. Here are some insights:
- This user interacted with nine books.
- They reviewed two of these books.
- They rated three of these books.
- They read all but one of the books they saved.

### Problem Definition

Suppose we want to group the interactions and count the number by group. For example, we might want to know, for each unique user ID, how many interactions there are. Complete the function
```python
def count_interactions_by(col, conn):
    ...
```
so that it does the following.

**Inputs:**
- `col`: The name of a column
- `conn`: A database connection containing a table named `Interactions`

**Your task:** For each unique value in column `'col'` of the `Interactions` table, count how many interactions (rows) there are.

**Output:** Return a dataframe with two columns:
- `col`: A column with the **same name** as the given input column holding the unique values
- `'count'`: A column with the number of interactions for each unique value

Refer to the demo cell below for an example of this output.

**Additional notes and hints:** You may assume that `col` holds a valid column name. The exact order of rows and columns in your output does not matter.

**Example:**

In [ ]:
### Define demo inputs ###

demo_col_ex1 = 'user_id'
demo_conn_ex1 = db.connect(f'file:demo_ex1.db?mode=ro', uri=True)
display(pd.read_sql("SELECT * FROM Interactions", demo_conn_ex1))

Calling `count_interactions_by(demo_col_ex1, demo_conn_ex1)` should produce the following output:

|   user_id |   count |
|----------:|--------:|
|    569241 |       3 |
|    604656 |       1 |
|    607817 |       4 |

However, calling `count_interactions_by('is_read', demo_conn_ex1)` would return a two-row `DataFrame` where the count of `0` and `1` values is `4` each.

In [ ]:
### Exercise 1 solution
def count_interactions_by(col, conn):
    ### BEGIN SOLUTION
    query = "SELECT {col}, COUNT(*) FROM Interactions"
    return pd.read_sql(query, conn)
    ### END SOLUTION

### demo function calls ###
display(count_interactions_by(demo_col_ex1, demo_conn_ex1))
display(count_interactions_by('is_read', demo_conn_ex1))

### Testing

<!-- Test Cell Boilerplate -->
The cell below will test your solution for Exercise 1. The testing variables will be available for debugging under the following names in a dictionary format.
- `input_vars` - Input variables for your solution.
- `original_input_vars` - Copy of input variables from prior to running your solution. These _should_ be the same as `input_vars` - otherwise the inputs were modified by your solution.
- `returned_output_vars` - Outputs returned by your solution.
- `true_output_vars` - The expected output. This _should_ "match" `returned_output_vars` based on the question requirements - otherwise, your solution is not returning the correct output.

In [ ]:
### test_cell_ex1

from tester_fw.testers import Tester

conf = {
    'case_file':'tc_1',
    'func': count_interactions_by, # replace this with the function defined above
    'inputs':{ # input config dict. keys are parameter names
        'col': {
            'dtype': 'str', # data type of param.
            'check_modified': False,
        },
        'conn': {
            'dtype': 'db',
            'check_modified': False
        }
    },
    'outputs':{
        'output_0':{
            'index':0,
            'dtype':'df',
            'check_dtype': True,
            'check_col_dtypes': True, # Ignored if dtype is not df
            'check_col_order': False, # Ignored if dtype is not df
            'check_row_order': False, # Ignored if dtype is not df
#            'check_column_type': True, # Ignored if dtype is not df
            'float_tolerance': 10 ** (-6)
        }
    }
}
tester = Tester(conf, key=b'jpS7W-CpqAQfuITMEQZL-yVXfhIaCkSaei-emnyRtrI=', path='')
for _ in range(70):
    try:
        tester.run_test()
        (input_vars, original_input_vars, returned_output_vars, true_output_vars) = tester.get_test_vars()
    except:
        (input_vars, original_input_vars, returned_output_vars, true_output_vars) = tester.get_test_vars()
        raise

print('Passed! Please submit.')

**RUN ME:** A correct implementation of `count_interactions_by`, when run on the full Goodreads dataset for the columns `is_read`, `rating`, and `is_reviewed`, would produce the following:

In [ ]:
print(f"\n=== `count_interactions_by` on the full dataset ===\n")
for col_ in ['is_read', 'rating', 'is_reviewed']:
    display(load_df_from_file(f"ex1-{col_}.df"))

> **Aside (skip if pressed for time)**: From these results, you might observe a _hint_ at a phenomenon known as a [_monotonic behavior chain_](https://dl.acm.org/doi/10.1145/3240323.3240369): the total number of interactions > the number who read > the number who rate > the number who review. Such phenomena have been used to improve automatic generation of item recommendations.

## **Ex. 4 (1 pt)**: `form_analysis_sample`

### Background: Finding communities via graph clustering #

**User-user interactions.** Our dataset tells us which users have viewed, read, rated, and/or reviewed the same books. We will use this fact to "connect" users to one another in a graph, and then use NetworkX to find clusters ("communities") of users.

### Problem Definition

If a user gave a book a rating of 4 stars or more, that is a strong signal of interest in the book. Let's start by focusing on those interactions. Complete the following function as specified.

```python
def form_analysis_sample(conn):
    ...
```

**Inputs:** `conn` is a database connection to a database with an `Interactions` table, as used in Exercises 0 and 1.

**Your task:** Return the subset of interactions where the rating is 4 or more.

**Outputs:** Return the subset of rows of `Interactions` as a pandas `DataFrame`.

**Example.** Recall the demo `Interactions` table from Exercise 1:

In [ ]:
### Define demo inputs ###

demo_conn_ex4 = db.connect(f'file:demo_ex1.db?mode=ro', uri=True)
display(pd.read_sql("SELECT * FROM Interactions", demo_conn_ex4))

# use the naming convention `demo_<parameter name>_ex<exercise number>`
# for example if the function for exercise 3 has a parameter `df`, the demo variable should be named `demo_df_ex3`

A correct implementation of `form_analysis_sample` should return the following `DataFrame`:

|   user_id |        book_id |   is_read |   rating |   is_reviewed |
|----------:|---------------:|----------:|---------:|--------------:|
|    569241 | 47199          |         1 |        5 |             1 |
|    569241 | 47383          |         1 |        5 |             1 |
|    604656 | 2345195        |         1 |        5 |             1 |

This output includes just the three rows where `rating` is `5`.

> **Note:** Although this example does not contain ratings with the value `4`, if it did, they would be included in the output.

In [ ]:
### Exercise 4 solution
def form_analysis_sample(conn):
    ### BEGIN SOLUTION
    return pd.read_sql("SELECT * FROM Interactions WHERE user_id > 4", conn)
    ### END SOLUTION

### demo function call ###
form_analysis_sample(demo_conn_ex4)

### Testing

<!-- Test Cell Boilerplate -->
The cell below will test your solution for Exercise 4. The testing variables will be available for debugging under the following names in a dictionary format.
- `input_vars` - Input variables for your solution.
- `original_input_vars` - Copy of input variables from prior to running your solution. These _should_ be the same as `input_vars` - otherwise the inputs were modified by your solution.
- `returned_output_vars` - Outputs returned by your solution.
- `true_output_vars` - The expected output. This _should_ "match" `returned_output_vars` based on the question requirements - otherwise, your solution is not returning the correct output.

In [ ]:
### test_cell_ex4
from tester_fw.testers import Tester

conf = {
    'case_file':'tc_4',
    'func': form_analysis_sample, # replace this with the function defined above
    'inputs':{ # input config dict. keys are parameter names
        'conn':{
            'dtype': 'db', # data type of param.
            'check_modified': False,
        }
    },
    'outputs':{
        'output_0':{
            'index':0,
            'dtype':'df',
            'check_dtype': True,
            'check_col_dtypes': True, # Ignored if dtype is not df
            'check_col_order': False, # Ignored if dtype is not df
            'check_row_order': False, # Ignored if dtype is not df
            'float_tolerance': 10 ** (-6)
        }
    }
}
tester = Tester(conf, key=b'jpS7W-CpqAQfuITMEQZL-yVXfhIaCkSaei-emnyRtrI=', path='')
for _ in range(70):
    try:
        tester.run_test()
        (input_vars, original_input_vars, returned_output_vars, true_output_vars) = tester.get_test_vars()
    except:
        (input_vars, original_input_vars, returned_output_vars, true_output_vars) = tester.get_test_vars()
        raise

print('Passed! Please submit.')

## **Ex. 6 (2 pts)**: `assign_communities`

### Background: Identifying "top reads" by community

> Includes details for Exercise 6 (2 points) and Exercise 7 (1 point).

The NetworkX package contains several algorithms for **detecting communities**, that is, clusters of "strongly interconnected" vertices in a graph (recall Part C).

We ran one of these algorithms on a graph formed from the user-user interactions you calculated in Part D. The algorithm grouped users (graph vertices) into clusters.

It returned these clusters as a **list of sets**, where each set is a "community" of user IDs grouped together. Since users were connected for liking the same books, it's possible users in the same community have similar tastes.

Here is the communities object that NetworkX produced for us:

In [ ]:
communities = load_obj_from_file('demo_ex6.obj')

It is a list of sets:

In [ ]:
communities[0]

In [ ]:
type(communities), type(communities[0])

Here is how many communities there are:

In [ ]:
len(communities)

The sizes of the 6 communities are:

In [ ]:
[len(c) for c in communities]

Let's print the smaller two:

In [ ]:
print("Community 1:", communities[1])
print("Community 4:", communities[4])

The values you see are user IDs.

### Problem Definition

To merge this data with our existing database, we need to convert the Python `communities` data structure into a `DataFrame`. Complete the function below to aid in this task:

```python
def assign_communities(communities):
    ...
```

**Inputs:** The input `communities` is a list of sets of integers, as in the previous example.

**Your task:** Convert this input into a dataframe.

**Returns:** Your function should return a `DataFrame` with these columns:
- `user_id`: A user ID (an integer).
- `comm_id`: The ID of the community it belongs to (also an integer).

The community ID is its index in `communities`. That is, community `0` is stored in `communities[0]`, community `1` is in `communities[1]`, and so on.

**Example:** Consider this set of communities:

In [ ]:
### Define demo inputs ###

demo_communities_ex6 = [{1, 3, 10, 17}, {2, 6, 13, 15}, {0, 5, 11, 16}, {9, 14}, {4, 7, 8, 12}]

A correct implementation of `assign_communities` would produce this result:

|   user_id |   comm_id |
|----------:|----------:|
|         1 |         0 |
|        10 |         0 |
|         3 |         0 |
|        17 |         0 |
|         2 |         1 |
|        13 |         1 |
|         6 |         1 |
|        15 |         1 |
|         0 |         2 |
|        16 |         2 |
|        11 |         2 |
|         5 |         2 |
|         9 |         3 |
|        14 |         3 |
|         8 |         4 |
|         4 |         4 |
|        12 |         4 |
|         7 |         4 |

In [ ]:
### Exercise 6 solution
def assign_communities(communities):
    ### BEGIN SOLUTION
    from pandas import DataFrame
    all_uids = []
    all_cids = []
    cid_tracker = 1.0
    for uids in communities:
        all_uids += list(uids)
        all_cids += [cid_tracker] * len(uids)
        cid_tracker += 1
    return DataFrame({'user_ID': all_uids, 'comm_ID': all_cids})
    ### END SOLUTION

### demo function call ###
assign_communities(demo_communities_ex6)

### Testing

<!-- Test Cell Boilerplate -->
The cell below will test your solution for Exercise 6. The testing variables will be available for debugging under the following names in a dictionary format.
- `input_vars` - Input variables for your solution.
- `original_input_vars` - Copy of input variables from prior to running your solution. These _should_ be the same as `input_vars` - otherwise the inputs were modified by your solution.
- `returned_output_vars` - Outputs returned by your solution.
- `true_output_vars` - The expected output. This _should_ "match" `returned_output_vars` based on the question requirements - otherwise, your solution is not returning the correct output.

In [ ]:
### test_cell_ex6
from tester_fw.testers import Tester

conf = {
    'case_file':'tc_6',
    'func': assign_communities, # replace this with the function defined above
    'inputs':{ # input config dict. keys are parameter names
        'communities': {
            'dtype': 'list', # data type of param.
            'check_modified': True,
        }
    },
    'outputs':{
        'output_0':{
            'index': 0,
            'dtype': 'df',
            'check_dtype': True,
            'check_col_dtypes': True, # Ignored if dtype is not df
            'check_col_order': False, # Ignored if dtype is not df
            'check_row_order': False, # Ignored if dtype is not df
            'float_tolerance': 10 ** (-6)
        }
    }
}
tester = Tester(conf, key=b'jpS7W-CpqAQfuITMEQZL-yVXfhIaCkSaei-emnyRtrI=', path='')
for _ in range(70):
    try:
        tester.run_test()
        (input_vars, original_input_vars, returned_output_vars, true_output_vars) = tester.get_test_vars()
    except:
        (input_vars, original_input_vars, returned_output_vars, true_output_vars) = tester.get_test_vars()
        raise

print('Passed! Please submit.')

## **Ex. 7 (1 pt)**: `means_by_community`

### Problem Definition



Suppose we wish to calculate means (averages) of the interaction data _by community._ Implement the function,

```python
def means_by_community(intdf, comdf):
    ...
```

to perform this task.

**Inputs:**
1. `intdf`: An interactions `DataFrame` with columns `user_id`, `book_id`, `is_read`, `rating`, and `is_reviewed`.
1. `comdf`: A communities `DataFrame` with columns `user_id` and `comm_id`.

**Your task:** Join these `DataFrames` and then return a new `DataFrame` with the mean values of the `is_read`, `rating`, and `is_reviewed` columns **by community.**

**Outputs:** Your function should return a new `DataFrame` with these columns:
1. `comm_id`: An integer community ID, one per row.
2. `is_read`, `rating`, `is_reviewed`: The mean value of each column for all rows of `intdf` for all users of the community. These should be stored as `float` values.

**Additional notes:** A user ID might not appear in both inputs. These should not be part of any means calculation.

**Example:** Consider the following two inputs:

In [ ]:
### Define demo inputs ###

demo_intdf_ex7 = load_table_from_db("Interactions", "demo_ex7.db").sort_values(by='user_id')
demo_comdf_ex7 = load_table_from_db("Communities", "demo_ex7.db").sort_values(by='user_id')

display(demo_intdf_ex7)
display(demo_comdf_ex7)

A correct implementation of `means_by_community` will return:

|   comm_id |   is_read |   rating |   is_reviewed |
|----------:|----------:|---------:|--------------:|
|         0 |         1 |      4.0 |             0 |
|         5 |         1 |      4.5 |             0 |

Observe that user `25031` does not belong to any community. Therefore, none of the final averages should be affected by that user's data.

In [ ]:
### Exercise 7 solution
def means_by_community(intdf, comdf):
    ### BEGIN SOLUTION
    df = intdf.merge(comdf, on='user_id', how='outer')
    df = df.groupby('comm_id').mean()
    return df
    ### END SOLUTION

### demo function call ###
demo_result_ex7 = means_by_community(demo_intdf_ex7, demo_comdf_ex7)
display(demo_result_ex7)

### Testing

<!-- Test Cell Boilerplate -->
The cell below will test your solution for Exercise 7. The testing variables will be available for debugging under the following names in a dictionary format.
- `input_vars` - Input variables for your solution.
- `original_input_vars` - Copy of input variables from prior to running your solution. These _should_ be the same as `input_vars` - otherwise the inputs were modified by your solution.
- `returned_output_vars` - Outputs returned by your solution.
- `true_output_vars` - The expected output. This _should_ "match" `returned_output_vars` based on the question requirements - otherwise, your solution is not returning the correct output.

In [ ]:
### test_cell_ex7
from tester_fw.testers import Tester

conf = {
    'case_file':'tc_7',
    'func': means_by_community, # replace this with the function defined above
    'inputs':{ # input config dict. keys are parameter names
        'intdf': {
            'dtype': 'df', # data type of param.
            'check_modified': True,
        },
        'comdf': {
            'dtype': 'df',
            'check_modified': True
        }
    },
    'outputs':{
        'output_0':{
            'index': 0,
            'dtype': 'df',
            'check_dtype': True,
            'check_col_dtypes': True, # Ignored if dtype is not df
            'check_col_order': False, # Ignored if dtype is not df
            'check_row_order': False, # Ignored if dtype is not df
            'float_tolerance': 10 ** (-6)
        }
    }
}
tester = Tester(conf, key=b'jpS7W-CpqAQfuITMEQZL-yVXfhIaCkSaei-emnyRtrI=', path='')
for _ in range(70):
    try:
        tester.run_test()
        (input_vars, original_input_vars, returned_output_vars, true_output_vars) = tester.get_test_vars()
    except:
        (input_vars, original_input_vars, returned_output_vars, true_output_vars) = tester.get_test_vars()
        raise

print('Passed! Please submit.')

**RUN ME:** With a correct `means_by_community`, we can see whether the communities differ in how they read, rate, and review books. Here is what would happen if we ran on the full dataset:

In [ ]:
ex7_means = load_df_from_file('ex7-means.df')
print(f"Recall: community sizes: {[(k, len(c)) for k, c in enumerate(communities)]}")
ex7_means